In [3]:
import io
from io import BytesIO
from zipfile import ZipFile
import pandas as pd
import requests
import pytz, datetime

from googletrans import Translator
translator = Translator()

In [4]:
required_columns = ['Périmètre', 'Nature', 'Date', 'Heures', 'Consommation',
       'Prévision J-1', 'Prévision J', 'Fioul', 'Charbon', 'Gaz', 'Nucléaire',
       'Eolien', 'Solaire', 'Hydraulique', 'Pompage', 'Bioénergies',
       'Ech. physiques', 'Taux de Co2', 'Ech. comm. Angleterre',
       'Ech. comm. Espagne', 'Ech. comm. Italie', 'Ech. comm. Suisse',
       'Ech. comm. Allemagne-Belgique', 'Fioul - TAC', 'Fioul - Cogén.',
       'Fioul - Autres', 'Gaz - TAC', 'Gaz - Cogén.', 'Gaz - CCG',
       'Gaz - Autres', 'Hydraulique - Fil de l?eau + éclusée',
       'Hydraulique - Lacs', 'Hydraulique - STEP turbinage',
       'Bioénergies - Déchets', 'Bioénergies - Biomasse',
       'Bioénergies - Biogaz']

In [6]:
url_list = ["https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_Annuel-Definitif_2017.zip",
            "https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_Annuel-Definitif_2018.zip",
            "https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_En-cours-Consolide.zip",
           "https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_En-cours-TR.zip"]

In [7]:
df_list = []

for url in url_list:
    content = requests.get(url)
    zf = ZipFile(BytesIO(content.content))

    for item in zf.namelist():
        print("File in zip: "+  item)

    # find the first matching csv file in the zip:
    match = [s for s in zf.namelist() if ".xls" in s][0]
    
    tmp_df = pd.read_table(zf.open(match), index_col=False, usecols = required_columns, encoding='ISO-8859-1').head(-1)
    
    df_list.append(tmp_df)
    
df = pd.concat(df_list).reset_index(drop=True)

translated_columns = [translator.translate(i, src='fr', dest='en').text for i in df.columns]
df.columns = translated_columns

#consider all datetime is in summer time of france
df['datetime'] = pd.to_datetime(df['Dated'] + " " + df['Hours']) - datetime.timedelta(hours=2)

File in zip: eCO2mix_RTE_Annuel-Definitif_2017.xls
File in zip: eCO2mix_RTE_Annuel-Definitif_2018.xls
File in zip: eCO2mix_RTE_En-cours-Consolide.xls
File in zip: eCO2mix_RTE_En-cours-TR.xls


In [12]:
df.Dated

0         2017-01-01
1         2017-01-01
2         2017-01-01
3         2017-01-01
4         2017-01-01
             ...    
124219    2020-07-17
124220    2020-07-17
124221    2020-07-17
124222    2020-07-17
124223    2020-07-17
Name: Dated, Length: 124224, dtype: object

In [13]:
df.Hours

0         00:00
1         00:15
2         00:30
3         00:45
4         01:00
          ...  
124219    22:45
124220    23:00
124221    23:15
124222    23:30
124223    23:45
Name: Hours, Length: 124224, dtype: object

In [11]:
local

<DstTzInfo 'Europe/Paris' LMT+0:09:00 STD>

In [9]:
df_copy = df.copy()

In [14]:
local = pytz.timezone ("Europe/Paris")

df['datetime'] = df['Dated'] + " " + df['Hours']

df['datetime'] = df['datetime'].apply(lambda x: local.localize(datetime.datetime.strptime(x, "%Y-%m-%d %H:%M"), is_dst=True).astimezone(pytz.utc))

df.datetime.value_counts()    # double count for some dates

2017-03-26 00:30:00+00:00    2
2019-03-31 00:30:00+00:00    2
2020-03-29 00:30:00+00:00    2
2017-03-26 00:15:00+00:00    2
2020-03-29 00:45:00+00:00    2
                            ..
2017-10-14 14:30:00+00:00    1
2018-07-26 01:15:00+00:00    1
2018-01-22 09:00:00+00:00    1
2019-10-05 09:30:00+00:00    1
2017-12-31 22:00:00+00:00    1
Name: datetime, Length: 124208, dtype: int64

In [30]:
df.datetime.value_counts().head(17)

2017-03-26 00:30:00+00:00    2
2019-03-31 00:30:00+00:00    2
2020-03-29 00:30:00+00:00    2
2017-03-26 00:15:00+00:00    2
2020-03-29 00:45:00+00:00    2
2019-03-31 00:00:00+00:00    2
2018-03-25 00:15:00+00:00    2
2018-03-25 00:00:00+00:00    2
2019-03-31 00:15:00+00:00    2
2017-03-26 00:00:00+00:00    2
2020-03-29 00:00:00+00:00    2
2017-03-26 00:45:00+00:00    2
2019-03-31 00:45:00+00:00    2
2018-03-25 00:45:00+00:00    2
2018-03-25 00:30:00+00:00    2
2020-03-29 00:15:00+00:00    2
2019-01-07 06:15:00+00:00    1
Name: datetime, dtype: int64

In [26]:
print(df.datetime.shape[0] - df.datetime.unique().shape[0])

16


In [73]:
df.apply(lambda x : x.datetime == df.datetime.value_counts().head(17).index)

AttributeError: 'Series' object has no attribute 'datetime'

In [49]:
df.iloc[113574:113579]

,Perimeter,Nature,Dated,Hours,Consumption,Forecast D-1,Forecast J,Fuel oil,Coal,Gas,...,Gas - Cogen.,Gas - CCG,Gas - Others,Hydraulic - Stream of water + lock,Hydraulics - Lakes,Hydraulics - STEP turbines,Bioenergies - Waste,Bioenergies - Biomass,Bioenergies - Biogas,datetime
113574,France,Données consolidées,2020-03-29,01:30,47184,45300,46400,97.0,10.0,2475.0,...,2231.0,126.0,119.0,4481.0,1606.0,0.0,557.0,321.0,322.0,2020-03-29 00:30:00+00:00
113575,France,Données consolidées,2020-03-29,01:45,NaN,45300,46400,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-29 00:45:00+00:00
113576,France,Données consolidées,2020-03-29,02:00,46904,45300,46400,98.0,10.0,2453.0,...,2232.0,103.0,118.0,4463.0,1680.0,0.0,556.0,326.0,320.0,2020-03-29 00:00:00+00:00
113577,France,Données consolidées,2020-03-29,02:15,NaN,45300,46400,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-29 00:15:00+00:00
113578,France,Données consolidées,2020-03-29,02:30,47184,45300,46400,97.0,10.0,2475.0,...,2231.0,126.0,119.0,4481.0,1606.0,0.0,557.0,321.0,322.0,2020-03-29 00:30:00+00:00


In [138]:
# localising timezone to Paris, but probematics. 

#local = pytz.timezone ("Europe/Paris")

#df['datetime'] = df['Dated'] + " " + df['Hours']

#df['datetime'] = df['datetime'].apply(lambda x: local.localize(datetime.datetime.strptime(x, "%Y-%m-%d %H:%M"), is_dst=True).astimezone(pytz.utc))

#df.datetime.value_counts()    # double count for some dates

In [217]:
forecast_df_list = []

for forecast in wf_list:
    
    hist_url = "https://ai4impact.org/P003/historical/" + forecast +".csv"

    r = requests.get(hist_url)
    data = r.content.decode('utf8').split("UTC\n")[1]
    hist_tmp_df = pd.read_csv(io.StringIO(data))
    
    current_url = "https://ai4impact.org/P003/" + forecast +".csv"

    r = requests.get(current_url)
    data = r.content.decode('utf8').split("UTC\n")[1]
    current_tmp_df = pd.read_csv(io.StringIO(data))
    
    tmp_df = pd.concat([hist_tmp_df,current_tmp_df]).reset_index(drop=True).rename(columns={'Speed(m/s)':forecast + '_Speed(m/s)', 'Direction (deg N)':forecast + '_Direction (deg N)'})
    
    tmp_df['datetime'] = pd.to_datetime(tmp_df['Time'].str.replace("UTC", ""))
    
    tmp_df = tmp_df.drop(columns=['Time'])
    
    forecast_df_list.append(tmp_df)
    
main_df = df.copy()

for i in forecast_df_list:
    main_df = main_df.merge(i, how='left', on='datetime')